In [6]:
import pandas as pd 
import re 
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
fake=pd.read_csv('Fake.csv')
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
true=pd.read_csv('True.csv')
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
stop_words=set(stopwords.words('english'))

In [10]:
def clean_text(text):
    text=re.sub(r'[^a-zA-Z]', ' ', text)
    text=text.lower().split()
    words=[word for word in text if word not in stop_words]
    return ' '.join(words)

In [11]:
fake['label']=0
true['label']=1

In [12]:
df=pd.concat([fake, true])
df=df.sample(frac=1).reset_index(drop=True)

In [13]:
df['text']=df['text'].apply(clean_text)
df

,title,text,subject,date,label
0,How Is Panama’s “Migrant Crisis” Giving A FREE...,let sink word time us borders open president p...,politics,"Aug 10, 2016",0
1,No decision yet on whether to go ahead with Mo...,reuters spanish constitutional court ruling su...,worldnews,"October 5, 2017",1
2,LEAKED EMAIL: Hillary’s Team Caught Bashing “B...,catholic sure offends fellow catholics would v...,politics,"Oct 11, 2016",0
3,ALABAMA Takes Bold Steps to Protect Confederat...,thank alabama protecting history good bad impo...,left-news,"May 21, 2017",0
4,BREAKING! WIKILEAKS EMAIL Shows Bill Clinton A...,hillary probably know anything one either http...,left-news,"Oct 12, 2016",0
...,...,...,...,...,...
44893,UK hate crimes surge on Brexit and militant at...,london reuters hate crimes britain surged high...,worldnews,"October 17, 2017",1
44894,RACIST LIBERAL REPORTER Arrested In Connection...,media democrat party coming pretty hard presid...,left-news,"Mar 3, 2017",0
44895,‘This Is A Nightmare’: AL City’s Black Reside...,black residents dothan alabama quite literally...,News,"January 7, 2016",0
44896,Trump's top defense and homeland officials to ...,berlin reuters u secretary defense james matti...,politicsNews,"January 25, 2017",1


**Tokenization ans Padding**

In [20]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_WORDS=10000
MAX_LEN=300

tokenizer=Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(df['text'])

sequences=tokenizer.texts_to_sequences(df['text'])

X=pad_sequences(sequences, maxlen=MAX_LEN)
y=df['label'].values

**Building LSTM and Bidirectional LSTM Models**

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

embedding_dim=100

def lstm_model():
    model= Sequential()
    model.add(Embedding(input_dim=MAX_WORDS, output_dim=embedding_dim, input_length=MAX_LEN))
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def bilstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=MAX_WORDS, output_dim=embedding_dim, input_length=MAX_LEN))
    model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

**Train and Evaluate Model**

In [16]:
%%time

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

model=bilstm_model()

result=model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

C:\Users\pavan\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
506/506 ━━━━━━━━━━━━━━━━━━━━ 561s 1s/step - accuracy: 0.8893 - loss: 0.2439 - val_accuracy: 0.9836 - val_loss: 0.0547
Epoch 2/5
506/506 ━━━━━━━━━━━━━━━━━━━━ 570s 1s/step - accuracy: 0.9853 - loss: 0.0464 - val_accuracy: 0.9866 - val_loss: 0.0404
Epoch 3/5
506/506 ━━━━━━━━━━━━━━━━━━━━ 574s 1s/step - accuracy: 0.9912 - loss: 0.0266 - val_accuracy: 0.9805 - val_loss: 0.0591
Epoch 4/5
506/506 ━━━━━━━━━━━━━━━━━━━━ 559s 1s/step - accuracy: 0.9952 - loss: 0.0156 - val_accuracy: 0.9866 - val_loss: 0.0429
Epoch 5/5
506/506 ━━━━━━━━━━━━━━━━━━━━ 567s 1s/step - accuracy: 0.9965 - loss: 0.0122 - val_accuracy: 0.9766 - val_loss: 0.0852
CPU times: total: 1h 54min 51s
Wall time: 47min 10s


In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

281/281 ━━━━━━━━━━━━━━━━━━━━ 16s 55ms/step - accuracy: 0.9817 - loss: 0.0695
Test Accuracy: 97.81%


In [19]:
model.save("bilstm_fake_news_model.keras")


from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json

token_json = tokenizer.to_json()
with open("tokenizer.json", "w") as f:
    f.write(token_json)